### Importing Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ply
import os
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB

In [3]:
import re,string,pprint
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords

C:\Users\adity\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



In [142]:
os.chdir("D:\Education\Data Science\Workspace_R")

### Importing the Dataset

In [173]:
df=pd.read_excel('./SCM_AUG_SEP_DATA.xlsx')
df.head()

,Bugged,Severity,Product Family,Product Area,Product ID,Product,Component,Sub Component,Product Version,Root Cause,SR,Known Issue,Reason Code,Summary,Status,Substatus,Last Updated On,Date Opened,Days Open,Date Closed
0,No,2-Significant,Oracle Platform Cloud,Internet of Things Cloud Service,11268,Internet of Things Cloud Service,Connected Worker,Generic Issues,18.2.3,Cloud-Implementation,1,No,"14 Undetermined,Cust Abandoned",can't use google from inside oracle network,Closed,Not Entitled,2018-08-03 01:39:45,2017-05-06 03:34:25,454,2018-08-02 19:39:28
1,No,2-Significant,Oracle Software Cloud,Oracle Enterprise Resource Planning Cloud,10402,Oracle Fusion Inventory Management Cloud Service,Financial Orchestration,Other Setup Tasks,11.12.1.0.0,Cloud-Post Go Live,2,Unspecified,12 Usability/Training-End User,Intercompany ap invoice accrual account is not...,Closed,Resolved with Solution,2018-08-17 12:42:31,2017-07-15 19:47:15,391,2018-08-10 10:07:55
2,No,2-Significant,Procurement,Procurement,10413,Oracle Fusion Purchasing Cloud Service,Purchasing Setup,Common Procurement Configuration,11.12.1.0.0,Cloud-Implementation,3,Yes,02 Enhancement Requests,"[IMPL SPT]Customer,BankOfAmerica wants to know...",Closed,Enhancement Request,2018-08-20 14:44:08,2017-12-04 05:26:42,252,2018-08-13 13:31:59
3,No,4-Minimal,Oracle Software Cloud,Oracle Enterprise Resource Planning Cloud,11179,Oracle Fusion Product Development Cloud Service,New Item Requests and Change Orders1,Manage Change Orders1,11.13.17.09.0,Cloud-Implementation,4,Yes,02 Enhancement Requests,[ER 27439011] Unable to create Multilist attri...,Closed,Enhancement Request,2018-08-29 18:41:07,2018-01-11 13:39:06,223,2018-08-22 17:30:20
4,No,4-Minimal,Oracle Software Cloud,Oracle Enterprise Resource Planning Cloud,11662,Oracle Fusion Cost Management Cloud Service,Landed Cost Management,Manage Trade Operations,11.13.18.02.0,Cloud-Post Go Live,5,Unspecified,07 Setup / Configuration,Error ORA-00001: unique constraint (FUSION.CML...,Closed,Resolved with Solution,2018-08-22 18:25:33,2018-02-09 14:44:46,187,2018-08-15 15:29:51


In [176]:
len(df)

9505

In [179]:
df=df.drop_duplicates().reset_index(drop=True)
len(df)

9505

In [180]:
df.columns=df.columns.astype(str)

In [181]:
df_txt=pd.DataFrame(df.loc[:,['Summary','Product Family']])

In [182]:
df_txt.head(50)

,Summary,Product Family
0,can't use google from inside oracle network,Oracle Platform Cloud
1,Intercompany ap invoice accrual account is not...,Oracle Software Cloud
2,"[IMPL SPT]Customer,BankOfAmerica wants to know...",Procurement
3,[ER 27439011] Unable to create Multilist attri...,Oracle Software Cloud
4,Error ORA-00001: unique constraint (FUSION.CML...,Oracle Software Cloud
5,"ER 27813084-Breaking out security policy ""Mana...",Oracle Software Cloud
6,Cost processor end in warning,Oracle Software Cloud
7,Import files succesfull but no Owner Spoke Sys...,Supply Chain Management
8,AutoInvoice not honoring LE Timezone Setting,Oracle Software Cloud
9,Unable to split requisition line in process re...,Procurement


In [183]:
df_txt['Summary']=df_txt['Summary'].apply(lambda x : str(x))

#### Removing the foreigh characters in our Text Summary

In [184]:
df_txt['Summary'] = df_txt['Summary'].str.replace(r'[^\x00-\x7F]+', '')

In [185]:
df_txt.head(50)

,Summary,Product Family
0,can't use google from inside oracle network,Oracle Platform Cloud
1,Intercompany ap invoice accrual account is not...,Oracle Software Cloud
2,"[IMPL SPT]Customer,BankOfAmerica wants to know...",Procurement
3,[ER 27439011] Unable to create Multilist attri...,Oracle Software Cloud
4,Error ORA-00001: unique constraint (FUSION.CML...,Oracle Software Cloud
5,"ER 27813084-Breaking out security policy ""Mana...",Oracle Software Cloud
6,Cost processor end in warning,Oracle Software Cloud
7,Import files succesfull but no Owner Spoke Sys...,Supply Chain Management
8,AutoInvoice not honoring LE Timezone Setting,Oracle Software Cloud
9,Unable to split requisition line in process re...,Procurement


In [186]:
df_txt['Summary'].replace('',np.nan,inplace=True)

In [187]:
df_txt.dropna(subset=['Summary'],inplace=True)

In [188]:
len(df_txt)

9415

In [146]:
df['Product Family'].value_counts()

Oracle Software Cloud                        6589
Procurement                                  2093
Supply Chain Management                       681
Oracle Platform Cloud                         137
Developer Tools                                 1
Oracle Enterprise Resource Planning Cloud       1
2476                                            1
Data Collection                                 1
Customer Relationship Management                1
Name: Product Family, dtype: int64

In [147]:
df_txt['Product Family'].unique()

array(['Oracle Platform Cloud', 'Oracle Software Cloud', 'Procurement',
       'Supply Chain Management'], dtype=object)

#### Removing the below less frequent Product Family Assignments

In [148]:
df_txt=df_txt[~df_txt['Product Family'].isin(['Customer Relationship Management','Data Collection','Developer Tools','Oracle Enterprise Resource Planning Cloud',2476])].reset_index(drop=True)

In [149]:
len(df_txt)

9410

#### Text Cleaning

In [150]:
df_txt1=df_txt['Summary']
df_txt1

0             can't use google from inside oracle network
1       Intercompany ap invoice accrual account is not...
2       [IMPL SPT]Customer,BankOfAmerica wants to know...
3       [ER 27439011] Unable to create Multilist attri...
4       Error ORA-00001: unique constraint (FUSION.CML...
5       ER 27813084-Breaking out security policy "Mana...
6                           Cost processor end in warning
7       Import files succesfull but no Owner Spoke Sys...
8            AutoInvoice not honoring LE Timezone Setting
9       Unable to split requisition line in process re...
10      Change order details report is not user friend...
11                      Planning central time zone in R13
12      Need to set up Credit check in R13 - dev Instance
13                How to customize Warehouse Manager role
14      Error occurred while displaying History tree t...
15      receipt accounting distribution entry show no ...
16      Updating of existing record not working using ...
17      Change

In [151]:
data=df_txt1.values.tolist()

In [152]:
data

["can't use google from inside oracle network",
 'Intercompany ap invoice accrual account is not populated',
 '[IMPL SPT]Customer,BankOfAmerica wants to know they can maintain duplicate Suppliers in Supplr Mgmt',
 '[ER 27439011] Unable to create Multilist attribute type on Change object',
 'Error ORA-00001: unique constraint (FUSION.CML_TOPT_CHARGE_REF_VALUES_U1) violated Trade Operation',
 'ER 27813084-Breaking out security policy "Manage Item Change Order" to handle independent privlleges',
 'Cost processor end in warning',
 'Import files succesfull but no Owner Spoke System at Child level',
 'AutoInvoice not honoring LE Timezone Setting',
 'Unable to split requisition line in process requistion screen...Getting error...',
 'Change order details report is not user friendly for changes with many affected objects',
 'Planning central time zone in R13',
 'Need to set up Credit check in R13 - dev Instance',
 'How to customize Warehouse Manager role',
 'Error occurred while displaying His

In [153]:
def sent_to_words(sentences,deacc=True):
    for i in sentences:
        yield(gensim.utils.simple_preprocess(str(i)))       

In [154]:
data_words=list(sent_to_words(data))

In [155]:
data_words

[['can', 'use', 'google', 'from', 'inside', 'oracle', 'network'],
 ['intercompany',
  'ap',
  'invoice',
  'accrual',
  'account',
  'is',
  'not',
  'populated'],
 ['impl',
  'spt',
  'customer',
  'bankofamerica',
  'wants',
  'to',
  'know',
  'they',
  'can',
  'maintain',
  'duplicate',
  'suppliers',
  'in',
  'supplr',
  'mgmt'],
 ['er',
  'unable',
  'to',
  'create',
  'multilist',
  'attribute',
  'type',
  'on',
  'change',
  'object'],
 ['error',
  'ora',
  'unique',
  'constraint',
  'fusion',
  'violated',
  'trade',
  'operation'],
 ['er',
  'breaking',
  'out',
  'security',
  'policy',
  'manage',
  'item',
  'change',
  'order',
  'to',
  'handle',
  'independent',
  'privlleges'],
 ['cost', 'processor', 'end', 'in', 'warning'],
 ['import',
  'files',
  'succesfull',
  'but',
  'no',
  'owner',
  'spoke',
  'system',
  'at',
  'child',
  'level'],
 ['autoinvoice', 'not', 'honoring', 'le', 'timezone', 'setting'],
 ['unable',
  'to',
  'split',
  'requisition',
  'line'

In [156]:
stop_words=stopwords.words('english')+list(string.punctuation)

In [157]:
def remove_stopwords(text):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text]

In [158]:
data_words_nostops=remove_stopwords(data_words)

In [159]:
data_words_nostops

[['use', 'google', 'inside', 'oracle', 'network'],
 ['intercompany', 'ap', 'invoice', 'accrual', 'account', 'populated'],
 ['impl',
  'spt',
  'customer',
  'bankofamerica',
  'wants',
  'know',
  'maintain',
  'duplicate',
  'suppliers',
  'supplr',
  'mgmt'],
 ['er',
  'unable',
  'create',
  'multilist',
  'attribute',
  'type',
  'change',
  'object'],
 ['error',
  'ora',
  'unique',
  'constraint',
  'fusion',
  'violated',
  'trade',
  'operation'],
 ['er',
  'breaking',
  'security',
  'policy',
  'manage',
  'item',
  'change',
  'order',
  'handle',
  'independent',
  'privlleges'],
 ['cost', 'processor', 'end', 'warning'],
 ['import',
  'files',
  'succesfull',
  'owner',
  'spoke',
  'system',
  'child',
  'level'],
 ['autoinvoice', 'honoring', 'le', 'timezone', 'setting'],
 ['unable',
  'split',
  'requisition',
  'line',
  'process',
  'requistion',
  'screen',
  'getting',
  'error'],
 ['change',
  'order',
  'details',
  'report',
  'user',
  'friendly',
  'changes',
  '

In [160]:
def lemmatization(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out=[]
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return texts_out

In [161]:
nlp=spacy.load('en',disable=['parser','ner'])
data_lemmatized=lemmatization(data_words_nostops,allowed_postags=['NOUN','ADJ','VERB','ADV'])

In [162]:
data_lemmatized

[['use', 'oracle', 'network'],
 ['intercompany', 'invoice', 'accrual', 'account', 'populate'],
 ['impl',
  'spt',
  'customer',
  'bankofamerica',
  'want',
  'know',
  'maintain',
  'duplicate',
  'supplier',
  'supplr',
  'mgmt'],
 ['unable', 'create', 'multilist', 'attribute', 'type', 'change', 'object'],
 ['error',
  'ora',
  'unique',
  'constraint',
  'fusion',
  'violate',
  'trade',
  'operation'],
 ['break',
  'security',
  'policy',
  'manage',
  'item',
  'change',
  'order',
  'handle',
  'independent',
  'privllege'],
 ['cost', 'processor', 'end', 'warning'],
 ['import',
  'file',
  'succesfull',
  'owner',
  'speak',
  'system',
  'child',
  'level'],
 ['honor', 'timezone', 'setting'],
 ['unable',
  'split',
  'requisition',
  'line',
  'process',
  'requistion',
  'screen',
  'get',
  'error'],
 ['change',
  'order',
  'detail',
  'report',
  'user',
  'friendly',
  'change',
  'many',
  'affected',
  'object'],
 ['plan', 'central', 'time', 'zone'],
 ['need', 'set', 'cre

In [163]:
Y=df_txt['Product Family']

In [164]:
len(Y)

9410

In [165]:
X=data_lemmatized
len(X)

9410

In [166]:
Y.unique()

array(['Oracle Platform Cloud', 'Oracle Software Cloud', 'Procurement',
       'Supply Chain Management'], dtype=object)

In [167]:
from sklearn.model_selection import train_test_split


In [168]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=200)

In [169]:
from sklearn.feature_extraction import text

In [170]:
tfidf=text.TfidfVectorizer(tokenizer=lambda i:i,lowercase=False,stop_words='english',max_features=3000,use_idf=True,ngram_range=(1,3))

In [119]:
X_tr=tfidf.fit_transform(X_train)
X_te=tfidf.transform(X_test)

In [120]:
Y_train.unique()


array(['Oracle Software Cloud', 'Procurement', 'Supply Chain Management',
       'Oracle Platform Cloud'], dtype=object)

In [121]:
Y_train

2094      Oracle Software Cloud
4044                Procurement
7667      Oracle Software Cloud
6961      Oracle Software Cloud
7398                Procurement
5095      Oracle Software Cloud
4648      Oracle Software Cloud
3224      Oracle Software Cloud
4772      Oracle Software Cloud
5747                Procurement
4369                Procurement
7564      Oracle Software Cloud
7552      Oracle Software Cloud
6696      Oracle Software Cloud
704                 Procurement
4835      Oracle Software Cloud
3920      Oracle Software Cloud
6336                Procurement
7695                Procurement
2969      Oracle Software Cloud
3439      Oracle Software Cloud
8234                Procurement
6836      Oracle Software Cloud
2624      Oracle Software Cloud
6159      Oracle Software Cloud
1690      Oracle Software Cloud
1885      Oracle Software Cloud
7149      Oracle Software Cloud
5563                Procurement
559       Oracle Software Cloud
                 ...           
462     

In [122]:
#Y_train.groupby(['Oracle Software Cloud','Procurement','Supply Chain Management','Oracle Platform Cloud','Data Collection','Customer Relationship Management']).count()

In [123]:
from sklearn.preprocessing import LabelEncoder

In [124]:
enc=LabelEncoder()

In [125]:
Y_tr=enc.fit_transform(Y_train)

In [126]:
Y_te=enc.transform(Y_test)

In [127]:
from sklearn.linear_model import LogisticRegression

In [128]:
clf=LogisticRegression(multi_class="multinomial",penalty='l2',solver='lbfgs')

In [129]:
import sklearn.model_selection as ms

In [130]:
param_grid={'C': np.arange(0,1,0.1)}

In [131]:
mod=ms.GridSearchCV(clf,param_grid=param_grid,n_jobs=-1,verbose=10)

In [132]:
mod.fit(X_tr,Y_tr)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:    4.5s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    4.6s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    5.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    5.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [133]:
mod.best_params_

{'C': 0.9}

In [134]:
mod.score(X_te,Y_te)

0.7848034006376196

In [117]:
Y_pred=mod.predict(X_te)

In [80]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_te,Y_pred)
cm

array([[  14,   13,    0,    0],
       [   0, 1229,   81,   17],
       [   0,  200,  212,    0],
       [   0,   99,    0,   17]], dtype=int64)

In [81]:
head_text='Vdi is having issue'

In [82]:
tfidf_headline=tfidf.transform([head_text])

In [83]:
mod.predict(tfidf_headline)

array([1], dtype=int64)

In [84]:
enc.inverse_transform(1)

'Oracle Software Cloud'

In [85]:
from sklearn import naive_bayes,svm,metrics,ensemble

In [86]:
def train_model(classifier,feature_vector_train,label,feature_vector_valid):
    classifier.fit(feature_vector_train,label)
    predictions=classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(predictions,Y_te)

In [87]:
nb=train_model(naive_bayes.MultinomialNB(),X_tr,Y_tr,X_te)


In [88]:
print(nb)

0.7656748140276302


In [89]:
RF=train_model(ensemble.RandomForestClassifier(),X_tr,Y_tr,X_te)

In [90]:
RF

0.7778958554729012

In [135]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_tr, Y_tr)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [136]:
Y_pr = classifier.predict(X_te)

In [95]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_te, Y_pr)
cm

array([[  17,    9,    1,    0],
       [   0, 1233,   74,   20],
       [   0,  195,  217,    0],
       [   0,   94,    0,   22]], dtype=int64)

In [138]:
cm.print_stats()

AttributeError: 'numpy.ndarray' object has no attribute 'print_stats'

In [96]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [98]:
ac=accuracy_score(Y_te,Y_pr)
ac

0.7911795961742827